In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

# Load your dataset
data = pd.read_csv('df_for_models.csv')

# Define selected features and target variable
selected_features = ['T (K)', 'P (MPa)', 'methane', 'ethane',
                     'propane', 'butane', 'pentane', 'ipentane', 'hexane',
                     'heptane', 'octane', 'nonane', 'decane', 'helium', 'oxygen',
                     'nitrogen', 'water', 'argon', 'hydrogen',
                     'H2S', 'CO2']
target_variable = 'z'

# Extract the selected features and target variable
X = data[selected_features]
y = data[target_variable]

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize and scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

best_r2 = -np.inf
best_num_hidden_neurons = 0

for num_hidden_neurons in range(1, 1001):
    # Randomly initialize and fix the hidden layer weights and biases
    num_input_features = X_train_scaled.shape[1]
    input_weights = np.random.rand(num_input_features, num_hidden_neurons) * 2 - 1
    hidden_biases = np.random.rand(1, num_hidden_neurons) * 2 - 1

    # Compute the hidden layer outputs
    def sigmoid_activation(x):
        return 1 / (1 + np.exp(-x))

    activation_func = sigmoid_activation
    hidden_outputs_train = activation_func(np.dot(X_train_scaled, input_weights) + hidden_biases)
    hidden_outputs_test = activation_func(np.dot(X_test_scaled, input_weights) + hidden_biases)

    # Randomly initialize the output layer weights
    output_weights = np.random.rand(num_hidden_neurons, 1) * 2 - 1

    # Calculate the predicted values for training and testing sets
    y_pred_train = np.dot(hidden_outputs_train, output_weights)
    y_pred_test = np.dot(hidden_outputs_test, output_weights)

    # Calculate R-squared for evaluation
    r2_train = r2_score(y_train, y_pred_train)
    r2_test = r2_score(y_test, y_pred_test)

    # Print the error per iteration
    print(f"Iteration {num_hidden_neurons}: R-squared (Train): {r2_train}, R-squared (Test): {r2_test}")

    # Update the best R-squared and number of hidden neurons if current R-squared is better
    if r2_test > best_r2:
        best_r2 = r2_test
        best_num_hidden_neurons = num_hidden_neurons

print("Best number of hidden neurons:", best_num_hidden_neurons)

# Randomly initialize and fix the hidden layer weights and biases using the best number of hidden neurons
input_weights = np.random.rand(num_input_features, best_num_hidden_neurons) * 2 - 1
hidden_biases = np.random.rand(1, best_num_hidden_neurons) * 2 - 1

# Compute the hidden layer outputs
hidden_outputs_train = activation_func(np.dot(X_train_scaled, input_weights) + hidden_biases)
hidden_outputs_test = activation_func(np.dot(X_test_scaled, input_weights) + hidden_biases)

# Randomly initialize the output layer weights
output_weights = np.random.rand(best_num_hidden_neurons, 1) * 2 - 1

# Calculate the predicted values for training and testing sets
y_pred_train = np.dot(hidden_outputs_train, output_weights)
y_pred_test = np.dot(hidden_outputs_test, output_weights)

# Calculate R-squared for evaluation
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

print("Final R-squared (Train):", r2_train)
print("Final R-squared (Test):", r2_test)

# Plotting predicted vs measured with different colors
fig, ax = plt.subplots(figsize=(10, 6))

# Set the limits of x-axis and y-axis from 0.5 to 1.4
ax.set_xlim(0.5, 1.4)
ax.set_ylim(0.5, 1.4)

# Assigning different colors for predicted and measured values for test data
predicted_colors_test = ['blue' if pred >= actual else 'blue' for pred, actual in zip(y_pred_test, y_test)]
predicted_colors_train = ['tomato' if pred >= actual else 'tomato' for pred, actual in zip(y_pred_train, y_train)]

# Scatter plot for predicted vs measured with different colors for test data
pred_actual_scatter_train = ax.scatter(y_train, y_pred_train, c=predicted_colors_train, marker=".", s=40, label='Train Data')
pred_actual_scatter_test = ax.scatter(y_test, y_pred_test, c=predicted_colors_test, marker=".", s=40, label='Test Data')
ax.plot([0.5, 1.4], [0.5, 1.4], color='black', linestyle='--', lw=1)

# Legend for distinguishing predicted and actual markers for test data
predicted_patch = mpatches.Patch(color='blue', label='Test Data')
predicted_train_patch = mpatches.Patch(color='tomato', label='Train Data')
handles = [predicted_patch, predicted_train_patch]
plt.legend(handles=handles)

# Annotation for R-squared values on the plot at the bottom right
ax.text(0.95, 0.05, f'R\u00b2 (Train) = {r2_train:.6f}', transform=ax.transAxes,
        fontsize=10, color='tomato', ha='right', va='bottom')
ax.text(0.95, 0.1, f'R\u00b2 (Test) = {r2_test:.6f}', transform=ax.transAxes,
        fontsize=10, color='blue', ha='right', va='bottom')

# Set plot title and labels
ax.set_title('Predicted vs Measured')
ax.set_xlabel('Measured Values')
ax.set_ylabel('Predicted Values')

plt.show()
